# Extracting mover positions

<img align="right" src="https://movingpandas.github.io/movingpandas/assets/img/movingpandas.png">

[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/movingpandas/movingpandas-examples/main?filepath=1-tutorials/3-extracting-mover-positions.ipynb)
[![IPYNB](https://img.shields.io/badge/view-ipynb-hotpink)](https://github.com/movingpandas/movingpandas-examples/blob/main/1-tutorials/3-extracting-mover-positions.ipynb)
[![HTML](https://img.shields.io/badge/view-html-green)](https://movingpandas.github.io/movingpandas-website/1-tutorials/3-extracting-mover-positions.html)

The following examples show how to find a mover's position at a certain time or for a certain time span.

In [ ]:
import pandas as pd
import geopandas as gpd
import movingpandas as mpd
import shapely as shp
import hvplot.pandas 

from geopandas import GeoDataFrame, read_file
from shapely.geometry import Point, LineString, Polygon
from datetime import datetime, timedelta
from holoviews import opts

import warnings
warnings.filterwarnings('ignore')

opts.defaults(opts.Overlay(active_tools=['wheel_zoom'], frame_width=500, frame_height=400))

mpd.show_versions()

First, let's create a basic trajectory:

In [ ]:
df = pd.DataFrame([
  {'geometry':Point(0,0), 't':datetime(2018,1,1,12,0,0)},
  {'geometry':Point(6,0), 't':datetime(2018,1,1,12,6,0)},
  {'geometry':Point(6,6), 't':datetime(2018,1,1,12,10,0)},
  {'geometry':Point(9,9), 't':datetime(2018,1,1,12,15,0)}
]).set_index('t')
gdf = GeoDataFrame(df, crs=31256)
toy_traj = mpd.Trajectory(gdf, 1)
toy_traj

In [ ]:
ax = toy_traj.plot()
gpd.GeoSeries(toy_traj.get_start_location()).plot(ax=ax, color='blue')
gpd.GeoSeries(toy_traj.get_end_location()).plot(ax=ax, color='red')

### Extracting a mover's position at a certain time


When we call this method, the resulting point is directly rendered:

In [ ]:
toy_traj.get_position_at(datetime(2018,1,1,12,6,0), method="nearest")    

To see its coordinates, we can look at the print output:

In [ ]:
print(toy_traj.get_position_at(datetime(2018,1,1,12,6,0), method="nearest"))

The method parameter describes what the function should do if there is no entry in the trajectory GeoDataFrame for the specified timestamp. 

For example, there is no entry at 2018-01-01 12:07:00

In [ ]:
toy_traj.df

In [ ]:
t = datetime(2018,1,1,12,7,0)
print(toy_traj.get_position_at(t, method="nearest"))
print(toy_traj.get_position_at(t, method="interpolated"))
print(toy_traj.get_position_at(t, method="ffill")) # from the previous row
print(toy_traj.get_position_at(t, method="bfill")) # from the following row

In [ ]:
point = toy_traj.get_position_at(t, method="interpolated")

ax = toy_traj.plot()
gpd.GeoSeries(point).plot(ax=ax, color='red', markersize=100)

### Extracting trajectory segments based on time 

First, let's extract the trajectory segment for a certain time period:

In [ ]:
segment = toy_traj.get_segment_between(datetime(2018,1,1,12,6,0), datetime(2018,1,1,12,12,0))
print(segment)

In [ ]:
ax = toy_traj.plot()
segment.plot(ax=ax, color='red', linewidth=5)

### Extracting trajectory segments based on geometry (i.e. clipping)

Now, let's extract the trajectory segment that intersects with a given polygon:

In [ ]:
xmin, xmax, ymin, ymax = 2, 8, -10, 5
polygon = Polygon([(xmin, ymin), (xmin, ymax), (xmax, ymax), (xmax, ymin), (xmin, ymin)])
polygon

In [ ]:
intersections = toy_traj.clip(polygon)
intersections

In [ ]:
ax = toy_traj.plot()
gpd.GeoSeries(polygon).plot(ax=ax, color='lightgray')
intersections.plot(ax=ax, color='red', linewidth=5, capstyle='round')